In [1]:
from IPython.display import display
import pandas as pd
dir = "./csv_results/final_results/"

def calculate_stats(path):
    data = pd.read_csv(path)
    df = pd.DataFrame(data)
    # Calculate mean and std for 'our' method for each alpha
    alphas = df['alpha'].dropna().unique()
    new_df_list = []

    for alpha in alphas:
        our_stats = df[(df['method'] == 'our') & (df['alpha'] == alpha)][['mia', 'forget', 'retain', 'time']].agg(['mean', 'std'])
        retrained_stats = df[df['method'] == 'retrained'][['mia', 'forget', 'retain', 'time']].agg(['mean', 'std'])

        # Calculate absolute differences
        abs_diff = (our_stats.loc['mean'] - retrained_stats.loc['mean']).abs()

        # Create new dataframe
        new_df = pd.DataFrame({
            'method': ['our'],
            'alpha': [alpha],
            'mia': [f"{our_stats.loc['mean', 'mia']:.2f} ± {our_stats.loc['std', 'mia']:.2f} ({abs_diff['mia']:.2f})"],
            'forget': [f"{our_stats.loc['mean', 'forget']:.2f} ± {our_stats.loc['std', 'forget']:.2f} ({abs_diff['forget']:.2f})"],
            'retain': [f"{our_stats.loc['mean', 'retain']:.2f} ± {our_stats.loc['std', 'retain']:.2f} ({abs_diff['retain']:.2f})"],
            'time': [f"{our_stats.loc['mean', 'time']:.2f} ± {our_stats.loc['std', 'time']:.2f}"],
            'avg_gap': [round(abs_diff[['mia', 'forget', 'retain']].mean(), 3)]
        })

        new_df_list.append(new_df)

    # Concatenate all the new dataframes
    our_final_df = pd.concat(new_df_list, ignore_index=True)

    # Calculate mean and std for remaining methods without alphas
    methods_without_alpha = df['method'].dropna().unique()
    methods_without_alpha = [method for method in methods_without_alpha if method != 'our']

    new_df_list = []
    for method in methods_without_alpha:
        method_stats = df[df['method'] == method][['mia', 'forget', 'retain', 'time']].agg(['mean', 'std'])
        retrained_stats = df[df['method'] == 'retrained'][['mia', 'forget', 'retain', 'time']].agg(['mean', 'std'])

        # Calculate absolute differences
        abs_diff = (method_stats.loc['mean'] - retrained_stats.loc['mean']).abs()

        # Create new dataframe
        new_df = pd.DataFrame({
            'method': [method],
            'alpha': [None],
            'mia': [f"{method_stats.loc['mean', 'mia']:.2f} ± {method_stats.loc['std', 'mia']:.2f} ({abs_diff['mia']:.2f})"],
            'forget': [f"{method_stats.loc['mean', 'forget']:.2f} ± {method_stats.loc['std', 'forget']:.2f} ({abs_diff['forget']:.2f})"],
            'retain': [f"{method_stats.loc['mean', 'retain']:.2f} ± {method_stats.loc['std', 'retain']:.2f} ({abs_diff['retain']:.2f})"],
            'time': [f"{method_stats.loc['mean', 'time']:.2f} ± {method_stats.loc['std', 'time']:.2f}"],
            'avg_gap': [round(abs_diff[['mia', 'forget', 'retain']].mean(), 3)]
        })

        new_df_list.append(new_df)

    # Concatenate all the new dataframes
    rest_final_df = pd.concat(new_df_list, ignore_index=True)
    
    # Rank both dataframes separately by avg_gap
    our_final_df = our_final_df.sort_values(by='avg_gap', ascending=True).reset_index(drop=True)
    rest_final_df = rest_final_df.sort_values(by='avg_gap', ascending=True).reset_index(drop=True)
    
    return our_final_df, rest_final_df

Instance-wise Unlearning with ViT

In [2]:
# ViT CIFAR-10 Instance-wise
our, rest = calculate_stats(dir + "vit-instances/vit-cifar-10.csv")
display(our, rest)

,method,alpha,mia,forget,retain,time,avg_gap
0,our,2.0,0.87 ± 0.01 (0.00),1.00 ± 0.00 (0.01),1.00 ± 0.00 (0.00),7.04 ± 0.03,0.004
1,our,4.0,0.87 ± 0.00 (0.01),1.00 ± 0.00 (0.01),1.00 ± 0.00 (0.00),7.07 ± 0.01,0.006
2,our,8.0,0.87 ± 0.00 (0.01),1.00 ± 0.00 (0.01),1.00 ± 0.00 (0.00),7.03 ± 0.03,0.006
3,our,16.0,0.86 ± 0.01 (0.02),1.00 ± 0.00 (0.01),1.00 ± 0.00 (0.00),7.05 ± 0.01,0.010
4,our,32.0,0.78 ± 0.01 (0.10),1.00 ± 0.00 (0.01),1.00 ± 0.00 (0.00),7.05 ± 0.01,0.037
5,our,64.0,0.65 ± 0.09 (0.22),1.00 ± 0.01 (0.01),0.99 ± 0.01 (0.01),7.04 ± 0.00,0.079
6,our,128.0,0.54 ± 0.01 (0.34),1.00 ± 0.00 (0.01),0.97 ± 0.00 (0.03),7.04 ± 0.01,0.127
7,our,1024.0,0.46 ± 0.14 (0.42),1.00 ± 0.00 (0.01),0.95 ± 0.00 (0.05),2.45 ± 0.00,0.160
8,our,512.0,0.46 ± 0.14 (0.42),1.00 ± 0.00 (0.01),0.95 ± 0.00 (0.05),2.45 ± 0.00,0.160
9,our,256.0,0.45 ± 0.14 (0.42),1.00 ± 0.00 (0.01),0.95 ± 0.00 (0.05),2.45 ± 0.00,0.161


,method,alpha,mia,forget,retain,time,avg_gap
0,retrained,None,0.88 ± 0.01 (0.00),0.99 ± 0.00 (0.00),1.00 ± 0.00 (0.00),76.54 ± 6.66,0.000
1,original,None,0.88 ± 0.00 (0.00),1.00 ± 0.00 (0.01),1.00 ± 0.00 (0.00),84.29 ± 9.71,0.004
2,scrub,None,0.88 ± 0.01 (0.01),1.00 ± 0.00 (0.01),1.00 ± 0.00 (0.00),16.81 ± 0.36,0.006
3,ssd,None,0.89 ± 0.00 (0.01),1.00 ± 0.00 (0.01),1.00 ± 0.00 (0.00),13.70 ± 0.25,0.008
4,finetune,None,0.90 ± 0.01 (0.03),0.99 ± 0.01 (0.00),1.00 ± 0.00 (0.00),11.36 ± 0.24,0.010
5,unsir,None,0.90 ± 0.00 (0.02),0.99 ± 0.01 (0.00),0.99 ± 0.01 (0.01),10.72 ± 0.21,0.011
6,amnesiac,None,0.83 ± 0.01 (0.04),0.99 ± 0.00 (0.00),1.00 ± 0.00 (0.00),12.87 ± 0.25,0.014
7,neggrad,None,0.91 ± 0.00 (0.03),1.00 ± 0.00 (0.01),1.00 ± 0.00 (0.00),12.66 ± 0.27,0.014
8,bad-teacher,None,0.80 ± 0.01 (0.07),0.97 ± 0.00 (0.02),0.97 ± 0.00 (0.03),8.19 ± 0.11,0.041


In [3]:
# ViT CIFAR-100 Instance-wise
our, rest = calculate_stats(dir + "vit-instances/vit-cifar-100.csv")
display(our, rest)

,method,alpha,mia,forget,retain,time,avg_gap
0,our,4.0,0.72 ± 0.01 (0.00),0.96 ± 0.00 (0.04),0.96 ± 0.00 (0.00),7.06 ± 0.01,0.014
1,our,8.0,0.71 ± 0.00 (0.01),0.96 ± 0.00 (0.04),0.96 ± 0.00 (0.00),7.05 ± 0.00,0.017
2,our,2.0,0.73 ± 0.00 (0.01),0.96 ± 0.01 (0.04),0.96 ± 0.00 (0.00),7.06 ± 0.01,0.018
3,our,16.0,0.67 ± 0.02 (0.05),0.95 ± 0.00 (0.03),0.95 ± 0.01 (0.01),7.05 ± 0.00,0.031
4,our,32.0,0.62 ± 0.01 (0.10),0.96 ± 0.00 (0.04),0.93 ± 0.00 (0.03),7.05 ± 0.01,0.058
5,our,1024.0,0.61 ± 0.01 (0.11),0.96 ± 0.00 (0.04),0.92 ± 0.00 (0.04),7.06 ± 0.01,0.064
6,our,512.0,0.61 ± 0.01 (0.11),0.96 ± 0.00 (0.04),0.92 ± 0.00 (0.04),7.06 ± 0.01,0.064
7,our,256.0,0.61 ± 0.01 (0.11),0.96 ± 0.00 (0.04),0.92 ± 0.00 (0.04),7.05 ± 0.00,0.064
8,our,128.0,0.61 ± 0.01 (0.11),0.96 ± 0.00 (0.04),0.92 ± 0.00 (0.04),7.05 ± 0.00,0.064
9,our,64.0,0.60 ± 0.01 (0.12),0.96 ± 0.00 (0.04),0.92 ± 0.00 (0.04),7.05 ± 0.01,0.066


,method,alpha,mia,forget,retain,time,avg_gap
0,retrained,None,0.72 ± 0.00 (0.00),0.92 ± 0.00 (0.00),0.96 ± 0.00 (0.00),115.08 ± 3.86,0.000
1,amnesiac,None,0.75 ± 0.01 (0.03),0.94 ± 0.01 (0.02),0.98 ± 0.01 (0.02),13.01 ± 0.47,0.022
2,finetune,None,0.76 ± 0.02 (0.04),0.94 ± 0.02 (0.02),0.97 ± 0.01 (0.01),11.40 ± 0.18,0.023
3,ssd,None,0.75 ± 0.00 (0.03),0.97 ± 0.01 (0.05),0.96 ± 0.00 (0.00),13.83 ± 0.37,0.026
4,scrub,None,0.75 ± 0.00 (0.03),0.97 ± 0.00 (0.05),0.96 ± 0.00 (0.00),16.92 ± 0.13,0.027
5,original,None,0.75 ± 0.00 (0.03),0.97 ± 0.00 (0.05),0.96 ± 0.00 (0.00),122.19 ± 1.45,0.027
6,unsir,None,0.78 ± 0.00 (0.06),0.94 ± 0.01 (0.02),0.95 ± 0.01 (0.01),10.75 ± 0.20,0.032
7,bad-teacher,None,0.65 ± 0.01 (0.07),0.90 ± 0.01 (0.02),0.91 ± 0.01 (0.05),8.05 ± 0.08,0.046
8,neggrad,None,0.81 ± 0.01 (0.09),0.98 ± 0.00 (0.06),0.98 ± 0.00 (0.02),12.79 ± 0.37,0.056


In [4]:
# Vit MUFAC


Class-wise Unlearning with ViT